## Lasttests von Machine Learning Modellen zur Qualitätssicherung
Dieses Notebook ist Teil von <a href='https://datenverknoten.de/mlops/lasttests-von-machine-learning-modellen-zur-qualitatssicherung/' target='_blank'>einem Artikel</a> auf www.datenverknoten.de.
<br>Quelle des verwendeten Datensatzes: https://www.kaggle.com/lirilkumaramal/heart-stroke

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from datetime import date

import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient

## Datenvorbereitung
Zunächst wird der Datensatz in ein Pandas DataFrame geladen. Es handelt sich hierbei um verschiedene Parameter aus dem Gesundheitsbereich, die zur Vorhersage eines 
Infarktes verwendet werden können. 

In [2]:
stroke_raw = pd.read_csv('rawdata/train_strokes.csv').drop(columns=['id'])

Die kategorischen Daten werden mit einem LabelEncoder in Zahlenwerte überführt. Dies verursacht potenziell Probleme, wie bereits im ersten Teil beschrieben wurde. Es wird hier nicht weiter darauf eingegangen. Durch das Encoding werden Zeilen, in denen sich ein NaN in irgendeiner der zu encodierenden Spalten befindet, ausgelassen.

In [3]:
labelencoder = LabelEncoder()
stroke_pre = stroke_raw.copy()
stroke_pre = stroke_pre.dropna()
stroke_pre['gender'] = labelencoder.fit_transform(stroke_pre['gender'])
stroke_pre['ever_married'] = labelencoder.fit_transform(stroke_pre['ever_married'])
stroke_pre['work_type'] = labelencoder.fit_transform(stroke_pre['work_type'])
stroke_pre['Residence_type'] = labelencoder.fit_transform(stroke_pre['Residence_type'])
stroke_pre['smoking_status'] = labelencoder.fit_transform(stroke_pre['smoking_status'])

Für das Training ist es hilfreich, wenn die Daten skaliert werden, da sonst der Einfluss einer deutlich größeren Variable, wie z.B. Alter, die Wichtigkeit der anderen Variablen negativ beeinflussen kann.

In [4]:
cols = stroke_pre.columns
stroke_pre_unscaled = stroke_pre.values
min_max_scaler = preprocessing.MinMaxScaler()
stroke_pre_scaled = min_max_scaler.fit_transform(stroke_pre_unscaled)
stroke_pre_scaled = pd.DataFrame(stroke_pre_scaled,columns=cols)

Da es unpraktisch wäre, alle möglichen Subsets der Daten mit stroke = 0 mit einer Größe von 548 zu erstellen und für das Training des Multilayer Perceptron zu nutzen, wird stattdessen ein <i>sliding window</i> verwendet. Dies läuft Instanz für Instanz über alle stroke = 0 Daten und pickt jeweils 548 dieser Instanzen heraus, kombiniert sie mit allen stroke = 1 Daten und erstellt somit einen ausgeglichenen Datensatz. Damit dies möglich wird, werden die beiden Klassen aufgeteilt.

In [5]:
stroke_0 = stroke_pre_scaled[stroke_pre_scaled['stroke']==0]
stroke_1 = stroke_pre_scaled[stroke_pre_scaled['stroke']==1]

In der nächsten Zelle wird das Multilayer Perceptron (MLP) trainiert. Dabei wird der zuvor beschriebene <i>sliding window</i> Ansatz verwendet. Für jedes Modell wird die Genauigkeit bestimmt. Das Modell, welches die höchste Genauigkeit besitzt, lässt sich nach Beendigung aller Trainingsdurchläufe in der Variable <i>best_clf</i> finden. 

In [6]:
best_clf = None
highest_accuracy = None

# Sliding window wie zuvor auch
for i in range(0,1000):
    if(i % 100 == 0):
        print(i)
    # Zusammensetzen des Datensatzes
    stroke_0_set = stroke_0.sample(n=len(stroke_1))
    attached_set = stroke_0_set.append(stroke_1)
    X = attached_set.drop(columns=['stroke'])
    y = attached_set['stroke']
    # Es wird ein Train-Test Split erstellt
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,random_state=2)

    # Die berechnete Accuracy soll auf die Eigenschaften des Trainings- und Testdatensatzes bezogen werden. 
    # Darum werden die Mittelwerte der einzelnen Features bestimmt.
    stats_1 = X_train.mean().to_dict()
    stats_2 = X_test.mean().to_dict()
    # Das MLP wird trainiert
    clf_mlp = MLPClassifier(random_state=1, max_iter=1500,hidden_layer_sizes=(5, 2))
    clf_mlp.fit(X_train, y_train)
    
    acc = clf_mlp.score(X_test,y_test)
    if(highest_accuracy == None or acc > highest_accuracy):
        highest_accuracy = acc
        best_clf = clf_mlp


0
100
200
300
400
500
600
700
800
900


Als nächstes wird angegeben, unter welcher Adresse die zentrale registry zu finden ist. In dieser werden Referenzen zu den erstellten und getrackten Modellen gespeichert. Es gibt verschiedene kompatible Datenbanktypen. Hier wird eine SQLite Datenbank verwendet. Die Modelle selber liegen als Datei im Verzeichnis vor, das beim Start des MLFlow Servers angegeben wird. Die Tracking URI bestimmt, wo der MLFlow Server gestartet wurde und wohin die Parameter geloggt werden sollen.

In [7]:
registry_uri = 'sqlite:///mlflow.db'
tracking_uri = 'http://localhost:5000'
 
mlflow.set_registry_uri(registry_uri)
mlflow.set_tracking_uri(tracking_uri)

Im nächsten Schritt werden nun die Genauigkeit und das Modell gelogt, damit die Genauigkeit im GUI des MLFlow Servers sichtbar ist. Das Modell wird direkt registriert. Ab diesem Moment kann es (im GUI von MLFlow) in den Status Staging oder Productive verschoben werden. 

In [9]:
with mlflow.start_run() as run:
    mlflow.log_param("Accuracy", highest_accuracy)
    mlflow.sklearn.log_model(
        sk_model = best_clf,
        artifact_path = "mlpmodel",
        registered_model_name = "mlp_model"   
    )
    
mlflow.end_run()

Successfully registered model 'mlp_model'.
2021/04/02 22:39:57 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: mlp_model, version 1
Created version '1' of model 'mlp_model'.


Hier wird die run_id abgefragt. Diese besteht aus einer langen Zeichenkette und in einem gleichnamigen Ordner werden die Modellartefakte gespeichert. Die id wird benötigt, um einen Webserver zu starten, welcher das Modell als Webservice bereitstellt.

In [10]:
runid = run.info.run_id

In [11]:
print(runid)

'0703fe95356f4133a05ed2ed696d2c0f'

Der Befehl zum Bereitstellen des Modells in einem Webserver würde dann wie folgt aussehen

In [ ]:
print('mlflow models serve -p 5005 -m mlruns/0/'+str(runid)+'/artifacts/mlpmodel --no-conda')

Hier ist noch zusätzlich gezeigt, wie in Python ein request an das bereitgestellte Modell gesendet werden kann. Dafür muss der Webserver gestartet sein, wie im vorherigen Befehl beschrieben.

In [12]:
import requests

host = '127.0.0.1'
port = '5005'

url = f'http://{host}:{port}/invocations'

headers = {
    'Content-Type': 'application/json',
}

http_data = '{"columns":["gender","age","hypertension","heart_disease","ever_married","work_type","Residence_type","avg_glucose_level","bmi","smoking_status"],"data":[[0.5,1.0,0.0,0.0,1.0,0.5,1.0,0.1519657685,0.1904761905,0.5]]}'

r = requests.post(url=url, headers=headers, data=http_data)

print(f'Predictions: {r.text}')

Predictions: [1.0]
